In [2]:
df = spark.read.format("json").load("data/flight-data/json/2015-summary.json")

In [3]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [7]:
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType


myManualSchema = StructType([
    StructField("DEST_COUNTRY_NAME",StringType(), True),
    StructField("ORIGIN_COUNTRY_NAME",StringType(), True),
    StructField("count",IntegerType(), False, metadata={"name":"darshil"}),
])

In [8]:
df = spark.read.format("json").schema(myManualSchema).load("data/flight-data/json/2015-summary.json")

In [9]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [24]:
from pyspark.sql.functions import col, column


In [31]:
df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

In [32]:
from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)

In [35]:
myRow[2]

1

In [36]:
df.createOrReplaceTempView("dfTable")

In [38]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [40]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [41]:
from pyspark.sql.functions import expr, col, column
df.select(
		  expr("DEST_COUNTRY_NAME"),
		  col("DEST_COUNTRY_NAME"),
		  column("DEST_COUNTRY_NAME")).show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



In [42]:
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)


+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [43]:
#or you can use alias on top the expr 
df.select(expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME")).show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [44]:
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



In [45]:
df.selectExpr(
    "*", # all original columns
    "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")\
    .show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [49]:
from pyspark.sql.functions import lit
df.select(expr("*"), lit(-999).alias("negative_999")).show(20)

+--------------------+-------------------+-----+------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|negative_999|
+--------------------+-------------------+-----+------------+
|       United States|            Romania|   15|        -999|
|       United States|            Croatia|    1|        -999|
|       United States|            Ireland|  344|        -999|
|               Egypt|      United States|   15|        -999|
|       United States|              India|   62|        -999|
|       United States|          Singapore|    1|        -999|
|       United States|            Grenada|   62|        -999|
|          Costa Rica|      United States|  588|        -999|
|             Senegal|      United States|   40|        -999|
|             Moldova|      United States|    1|        -999|
|       United States|       Sint Maarten|  325|        -999|
|       United States|   Marshall Islands|   39|        -999|
|              Guyana|      United States|   64|        -999|
|       

In [50]:
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [52]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").show()

+--------------------+-------------------+-----+
|                dest|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [57]:
df.withColumn("count2", col("count") + 50).show()

+--------------------+-------------------+-----+------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|count2|
+--------------------+-------------------+-----+------+
|       United States|            Romania|   15|    65|
|       United States|            Croatia|    1|    51|
|       United States|            Ireland|  344|   394|
|               Egypt|      United States|   15|    65|
|       United States|              India|   62|   112|
|       United States|          Singapore|    1|    51|
|       United States|            Grenada|   62|   112|
|          Costa Rica|      United States|  588|   638|
|             Senegal|      United States|   40|    90|
|             Moldova|      United States|    1|    51|
|       United States|       Sint Maarten|  325|   375|
|       United States|   Marshall Islands|   39|    89|
|              Guyana|      United States|   64|   114|
|               Malta|      United States|    1|    51|
|            Anguilla|      United States|   41|

In [59]:
df.filter(col("count") < 2).show()


+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Croatia|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
|               Malta|      United States|    1|
|       United States|          Gibraltar|    1|
|Saint Vincent and...|      United States|    1|
|            Suriname|      United States|    1|
|       United States|             Cyprus|    1|
|        Burkina Faso|      United States|    1|
|            Djibouti|      United States|    1|
|       United States|            Estonia|    1|
|              Zambia|      United States|    1|
|              Cyprus|      United States|    1|
|       United States|          Lithuania|    1|
|       United States|           Bulgaria|    1|
|       United States|            Georgia|    1|
|       United States|            Bahrain|    1|
|       Cote d'Ivoir

In [60]:
df.where("count < 2").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [61]:
df.rdd.getNumPartitions() # 1

1

In [62]:
df.repartition(5)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]